In [ ]:
# Purpose: The following is a batch program to get sentiment class and summarized key points from OpenAI ChatGPT  
#          given a spreadsheet with records of customer feedback in text. 
#          This is an example program used for my Casualty Actuarial Society 2023 Annual Meeting presentation  
#          NPS_rawgrade is customer satisfaction score, which is optional. It is included to allow for assocation analysis
#          between output sentiment and satisfaction store. 
# By:      Frank Zhang - June 2023

In [ ]:
import os
import re as re
import openai
import pandas as pd
import numpy as np
 
openai.api_key = "_api_key_for_your_openai_account_"

# example of model_type: "text-davinci-003" or "gpt-4-0613"
# temperature 0-1, with lower value for more focused, coherent, and conservative output
def run_model(prompt, model_type):
  response = openai.Completion.create(
    model=model_type,
    prompt = prompt,
    temperature=0.3,   
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  
  return response['choices'][0].message.content
  
 
def loop_all(df_in, model_type):
   sentiment = []
   topic = []

   for i in range(len(df_in.index)):
      comment = df_in.iloc[i]["comments"]
      last_char = [".", "?", "!"] 
      if not (comment[-1] in last_char):
         comment=comment+"."

      try:
         sentiment_prompt = "[Classify the following paragraph into one of the following five sentiments - very negative, negative, neutral, positive, and very positive:] \n " + comment 
         topic_promt = "[Without auto-completing text, list one to three main topics discussed in the following paragraph:]\n" + comment

         sentiment_res = ""
         topic_res = ""
        
         sentiment_res = run_model(sentiment_prompt, model_type) 

         if len((df_in.iloc[i]["comments"]).split())>=3:
            topic_res = run_model(topic_promt, model_type) 
 
         sentiment.append(sentiment_res)
         topic.append(topic_res)

         if (i % 100==0):
            print("... Proceeding record  "+str(i))
      except:
         sentiment.append("")
         topic.append("")
         print("***An error occured processing record "+str(i) + "\n" + df_in.iloc[i]["comments"])

         pass
 
   df_out = df_in
   df_out['sentiment'] = sentiment
   df_out['topic'] = topic

   return(df_out)

In [2]:
df_comment_qtrly = pd.read_excel("./df_comment_2023q3.xlsx") # removed empty comments or with the following values: N/A NA, None, margin comment: none
print(df_comment_qtrly.shape)

(1171, 3)


,repond_id,nps_rawgrade,comments
0,203950,9.0,Great job all around.
1,203949,10.0,Let me know what was done with the girl that h...
2,203945,10.0,Mail check directly to lender (Regions Mortgag...
3,203944,10.0,No need to improve - they did a good job. Than...
4,203943,10.0,Overall we were very happy! But - I don't unde...


In [3]:
df_comment_qtrly["nps_rawgrade_category"] = pd.cut(df_comment_qtrly["nps_rawgrade"], [-1,6,8,10], labels=["Detractors","Neutrals", "Promoters"])

In [4]:
df_in = df_comment_qtrly
#df_comment_with_gpt_sentiment = loop_all(df_in,"gpt-4")    # https://platform.openai.com/account/rate-limits -- reaching limit
 

In [8]:
df_comment_with_gpt_sentiment = loop_all(df_in, "text-davinci-003") 

... Proceeding record  0
... Proceeding record  100
... Proceeding record  200
... Proceeding record  300
... Proceeding record  400
... Proceeding record  500
... Proceeding record  600
... Proceeding record  700
... Proceeding record  800
... Proceeding record  900
... Proceeding record  1000
... Proceeding record  1100


In [10]:
# df_comment_with_gpt_sentiment.to_csv("junk.csv")

df_comment_with_gpt_sentiment.to_csv("df_comment_with_gpt_sentiment_2023q3.csv")
